In [40]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

import os

from tensorflow.keras import datasets, layers, models, optimizers

from ImagingSoftware.ImageProcessing import *
#from ImagingSoftware.ImageClassifier import *

In [44]:
def make_dataset(exposure_time, n_tweezers, n_loops, decimate=2):
    path = f"Data/06:23:23/imaging exposure time = {exposure_time}ms/"
    n_files = len(os.listdir(path))
    skip = np.random.randint(n_files)
    training_crops = []
    training_labels = []
    testing_crops = []
    testing_labels = []
    positions = None
    for i, file in enumerate(os.listdir(path)):
        if file.endswith('.mat'):
            data = loadmat(path + file)
            if i == 0:
                processor = GreenImageProcessor(data['stack'], n_tweezers, n_loops)
                positions = processor.tweezer_positions
            else:
                processor = GreenImageProcessor(data['stack'], n_tweezers, n_loops, tweezer_positions=positions)
            crops, labels = processor.make_dataset()
            if i == skip:
                testing_crops.append(crops)
                testing_labels.append(labels)
            else:
                training_crops.append(crops)
                training_labels.append(labels)
    training_crops = np.concatenate(training_crops, axis=0)
    training_labels = np.concatenate(training_labels, axis=0)
    testing_crops = np.concatenate(testing_crops, axis=0)
    testing_labels = np.concatenate(testing_labels, axis=0)
    keep_training = np.random.choice(training_labels.shape[0], training_labels.shape[0] // 10 ** decimate, replace=False)
    keep_testing = np.random.choice(testing_labels.shape[0], testing_labels.shape[0] // 10 ** decimate, replace=False)
    np.savez_compressed(path + f"{exposure_time}ms dataset", training_crops=training_crops[keep_training],
                        training_labels=training_labels[keep_training],
                        testing_crops=testing_crops[keep_testing],
                        testing_labels=testing_labels[keep_testing])
    return

def load_dataset(exposure_time):
    path = f"Data/06:23:23/imaging exposure time = {exposure_time}ms/"
    return np.load(path + f"{exposure_time}ms dataset.npz")

In [45]:
def architecture_0(input_shape, n_filters, filter_size):
    network = models.Sequential()
    network.add(layers.Input(shape=(*input_shape, 1)))
    network.add(layers.Conv2D(n_filters, filter_size, strides=1, activation='relu'))
    network.add(layers.MaxPooling2D(pool_size=(2, 2)))
    network.add(layers.Flatten())
    network.add(layers.Dense(2, activation='sigmoid'))
    return network

def architecture_1(input_shape, n_filters, filter_size):
    print(type(filter_size[0]), filter_size[1])
    network = models.Sequential()
    network.add(layers.Input(shape=(*input_shape, 1)))
    network.add(layers.Conv2D(n_filters[0], filter_size[0], strides=1, activation='relu'))
    network.add(layers.Conv2D(n_filters[1], filter_size[1], strides=1, activation='relu'))
    network.add(layers.Flatten())
    network.add(layers.Dense(2, activation='sigmoid'))
    return network   

def architecture_2(input_shape, n_filters, filter_size):
    network = models.Sequential()
    network.add(layers.Input(shape=(*input_shape, 1)))
    network.add(layers.Conv2D(n_filters[0], filter_size[0], strides=1, activation='relu'))
    network.add(layers.MaxPooling2D(pool_size=(2, 2)))
    network.add(layers.Conv2D(n_filters[1], filter_size[1], strides=1, activation='relu'))
    network.add(layers.Flatten())
    network.add(layers.Dense(2, activation='sigmoid'))
    return network

def architecture_3(input_shape, n_filters, filter_size):
    network = models.Sequential()
    network.add(layers.Input(shape=(*input_shape, 1)))
    network.add(layers.Conv2D(n_filters[0], filter_size[0], strides=1, activation='relu'))
    network.add(layers.MaxPooling2D(pool_size=(2, 2)))
    network.add(layers.Conv2D(n_filters[1], filter_size[1], strides=1, activation='relu'))
    network.add(layers.Flatten())
    network.add(layers.Dense(9, activation='sigmoid'))
    network.add(layers.Dense(2, activation='sigmoid'))
    return network

# Make a File of Training/Testing Data


In [46]:
exposure_time = 5
n_tweezers = 100
n_loops = 20
make_dataset(exposure_time, n_tweezers, n_loops)

[array([[[103,  97, 102, ..., 111, 108, 101],
        [104, 109, 113, ...,  97, 105, 105],
        [100, 102, 119, ..., 106, 106, 107],
        ...,
        [107, 109, 104, ..., 103, 112, 110],
        [ 99, 110, 107, ...,  98, 105,  99],
        [105, 102, 105, ..., 117, 104,  98]],

       [[101,  96, 113, ..., 103, 105, 105],
        [106,  98, 103, ..., 113, 115,  99],
        [103, 115, 104, ..., 100, 101, 108],
        ...,
        [111, 106, 108, ...,  98, 101, 106],
        [104, 102, 108, ..., 110, 105, 104],
        [103, 101, 105, ...,  97,  98, 107]],

       [[108, 106,  96, ..., 103, 102, 104],
        [106, 102, 103, ..., 109, 116, 100],
        [115, 114, 114, ...,  98, 105, 102],
        ...,
        [102, 108, 101, ...,  90,  99, 100],
        [103, 119, 119, ..., 108, 102,  98],
        [110, 101, 118, ..., 102, 104, 109]],

       ...,

       [[105, 111, 101, ..., 114, 106,  98],
        [ 97, 118,  99, ..., 110, 104, 105],
        [104, 105, 112, ..., 100, 104,  9

# Load a File of Testing/Training Data

In [47]:
data = load_dataset(exposure_time)

# Make Neural Network Model

In [48]:
model = architecture_2(data['training_crops'].shape[1:], [10, 10], [3, 3])
data['training_crops'].shape

(22130, 16, 16)

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(data['training_crops'], data['training_labels'], epochs=8)

Epoch 1/8
70942/70942 [==============================] - 213s 3ms/step - loss: 0.2661 - binary_accuracy: 0.8841
Epoch 2/8
36904/70942 [==============>...............] - ETA: 2:13 - loss: 0.2553 - binary_accuracy: 0.8853

KeyboardInterrupt: 